In [1]:
import ondemand_dask

In [4]:
cluster_name = 'dask-test'
image_name = 'dask-build'
project = 'project'
cpu = 2
ram = 4096
zone = 'asia-southeast1-a'
worker_size = 4
webhook = 'https://hooks.slack.com/services/'

In [5]:
%%time

ip = ondemand_dask.spawn(cluster_name = cluster_name,
                   image_name = image_name,
                   project = project,
                   cpu = cpu,
                   ram = ram,
                   zone = zone,
                   worker_size = worker_size,
                   webhook = webhook)

/usr/local/lib/python3.7/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


Waiting instance `dask-test` to run.
Done.
35.247.140.249 10.148.0.120 done.
Waiting Dask cluster to run.
Done.
CPU times: user 208 ms, sys: 41 ms, total: 249 ms
Wall time: 1min 16s


In [6]:
from dask.distributed import Client

client = Client('%s:8786' % (ip['ip']))
client

Client Scheduler: tcp://35.247.140.249:8786 Dashboard: http://35.247.140.249:8787/status,Cluster Workers: 4 Cores: 4 Memory: 4.13 GB


In [7]:
import numpy as np
import dask.array as da

x = da.random.normal(10, 0.1, size=(50000, 50000),
                              chunks=(1000, 1000))
y = x.mean(axis=0)[::100]        

In [8]:
y

dask.array<getitem, shape=(500,), dtype=float64, chunksize=(10,), chunktype=numpy.ndarray>

In [9]:
%%time

y.compute()

CPU times: user 1.34 s, sys: 140 ms, total: 1.48 s
Wall time: 2min 23s


array([10.00011132, 10.00091399,  9.99978986,  9.99936512,  9.99944298,
        9.99947176,  9.9994397 ,  9.99978413,  9.99948791,  9.99960109,
       10.00006987,  9.99892816, 10.00010392, 10.00015213,  9.99990917,
        9.99932174,  9.99965898,  9.99958658, 10.00070288, 10.00023347,
       10.0008145 , 10.00083386,  9.99997893,  9.9994796 ,  9.99947307,
       10.0001795 ,  9.99922474,  9.99968583,  9.9991933 , 10.00023784,
       10.00016062,  9.99975801, 10.00001432,  9.99951411,  9.99949045,
       10.0003479 , 10.00120583, 10.00060394,  9.99975897,  9.99940395,
       10.0004124 ,  9.99948699, 10.00007405, 10.00073001,  9.99866462,
       10.00017047, 10.00052903, 10.00007267, 10.0000132 , 10.00105763,
        9.99926468,  9.99922943,  9.99960163,  9.99968009,  9.99974724,
       10.00037593,  9.99990911, 10.00070745,  9.99962045, 10.00036178,
       10.00005896, 10.00029212, 10.00009848, 10.0002518 , 10.00029585,
        9.99937951, 10.00063914, 10.00034953, 10.00067474,  9.99

## manually delete the cluster

If you finished the computation and decided no longer need it, you can manually delete it. Or simply just wait gracefully delete itself.

In [11]:
ondemand_dask.delete(cluster_name = cluster_name, project = project, zone = zone)